In [1]:
import numpy as np 
from numpy import *
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [2]:
from numpy import *
from numpy import linalg as la

def loadExData():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]
    
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]
    

In [2]:
U, Sigma, VT= linalg.svd([[1,1], [7,7]])

In [3]:
U

array([[-0.14142136, -0.98994949],
       [-0.98994949,  0.14142136]])

In [4]:
Sigma

array([1.00000000e+01, 2.82797782e-16])

In [5]:
VT

array([[-0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678]])

In [6]:
Data = loadExData()

In [7]:
U, Sigma, VT= linalg.svd(Data)

In [8]:
Sigma

array([9.64365076e+00, 5.29150262e+00, 6.49628424e-16, 1.43063514e-16,
       2.79192092e-17])

In [9]:
Sig3 = mat([
    [Sigma[0], 0, 0],
    [0, Sigma[1], 0],
    [0, 0, Sigma[2] ]
])

In [10]:
Sig3

matrix([[9.64365076e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 5.29150262e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 6.49628424e-16]])

### 用3维数据表示原来5维的数据

In [12]:
res = U[:,:3] * Sig3 * VT[:3,:]

In [16]:
np.around(res.getA(), decimals=1)

array([[ 0., -0., -0.,  2.,  2.],
       [ 0., -0., -0.,  3.,  3.],
       [ 0., -0., -0.,  1.,  1.],
       [ 1.,  1.,  1., -0., -0.],
       [ 2.,  2.,  2., -0., -0.],
       [ 5.,  5.,  5., -0., -0.],
       [ 1.,  1.,  1., -0., -0.]])

In [17]:
Data

[[0, 0, 0, 2, 2],
 [0, 0, 0, 3, 3],
 [0, 0, 0, 1, 1],
 [1, 1, 1, 0, 0],
 [2, 2, 2, 0, 0],
 [5, 5, 5, 0, 0],
 [1, 1, 1, 0, 0]]

# 相似度计算

In [3]:
def ecludSim(inA,inB):
    return 1.0/(1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [4]:
myMat = matrix(loadExData())

In [5]:
ecludSim(myMat[:,0], myMat[:,4])

0.12973190755680383

In [6]:
cosSim(myMat[:,0], myMat[:,4])

0.5

In [7]:

def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item].A>0, \
                                      dataMat[:,j].A>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])
        print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [15]:
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    U,Sigma,VT = la.svd(dataMat)
    Sig4 = mat(eye(4)*Sigma[:4]) #arrange Sig4 into a diagonal matrix
    xformedItems = dataMat.T * U[:,:4] * Sig4.I  #create transformed items
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [28]:
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1]#find unrated items 
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

In [29]:
myMat = mat(loadExData())

In [36]:
myMat[0,1]=myMat[0,0] = myMat[1,0]=myMat[2,0]=4
myMat[3,3]=2
recommend(myMat,2)


the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 0.928746
the 1 and 4 similarity is: 1.000000
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 1.000000
the 2 and 4 similarity is: 0.000000


[(2, 2.5), (1, 2.0243290220056256)]

In [38]:
recommend(myMat,2, simMeas=pearsSim)

the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 1.000000
the 1 and 4 similarity is: 1.000000
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 1.000000
the 2 and 4 similarity is: 0.000000


[(2, 2.5), (1, 2.0)]

In [39]:
recommend(myMat,2, simMeas=ecludSim)

the 1 and 0 similarity is: 1.000000
the 1 and 3 similarity is: 0.309017
the 1 and 4 similarity is: 0.333333
the 2 and 0 similarity is: 1.000000
the 2 and 3 similarity is: 0.500000
the 2 and 4 similarity is: 0.000000


[(2, 3.0), (1, 2.8266504712098603)]